In [ ]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []


    def word_counter_forward(self , length , dictionary):


      unigram_count = collections.Counter()
      bigram_count = collections.Counter()
      trigram_count = collections.Counter()
      fourgram_count = collections.Counter()
      fivegram_count = collections.Counter()


      for word in dictionary:
        if len(word) != length:
          continue
        # Unigram

        for letter in word:
          unigram_count[letter]+=1

        # Bigram

        bigram_word = '#' + word

        bigram_list = zip(bigram_word[:-1] , bigram_word[1:])

        for bigram in bigram_list:

          letter1 , letter2 = bigram
          bigram_count[letter1+letter2] += 1

        # Trigram

        trigram_word = '##' + word

        trigram_list = zip(trigram_word[:-2] , trigram_word[1:-1] , trigram_word[2:])

        for trigram in trigram_list:

          letter1 , letter2 , letter3 = trigram
          trigram_count[letter1 + letter2 + letter3] += 1

        # fourgram +  Fivegram

        fourgram_word = '####' + word
        fivegram_word = '####' + word

        fourgram_list = zip(fourgram_word[:-3] , fourgram_word[1:-2] , fourgram_word[2:-1] , fourgram_word[3:])

        for fourgram in fourgram_list:
          letter1 , letter2 , letter3 , letter4 = fourgram

          fourgram_count[letter1+letter2+letter3+letter4] += 1

        fivegram_list = zip(fivegram_word[:-4] , fivegram_word[1:-3] , fivegram_word[2:-2] , fivegram_word[3:-1] , fivegram_word[4:])

        for fivegram in fivegram_list:

          letter1 , letter2 , letter3 , letter4 , letter5 = fivegram

          fivegram_count[letter1 + letter2 + letter3 + letter4 + letter5] += 1


      return unigram_count , bigram_count , trigram_count , fourgram_count , fivegram_count


    def word_counter_backward(self , length , dictionary):


      unigram_count = collections.Counter()
      bigram_count = collections.Counter()
      trigram_count = collections.Counter()
      fourgram_count = collections.Counter()
      fivegram_count = collections.Counter()


      for word in dictionary:
        if len(word) != length:
          continue
        # Unigram

        for letter in word:
          unigram_count[letter]+=1

        # Bigram

        bigram_word =  word + '#'

        bigram_list = zip(bigram_word[:-1] , bigram_word[1:])

        for bigram in bigram_list:

          letter1 , letter2 = bigram
          bigram_count[letter1+letter2] += 1

        # Trigram

        trigram_word = word + '##'

        trigram_list = zip(trigram_word[:-2] , trigram_word[1:-1] , trigram_word[2:])

        for trigram in trigram_list:

          letter1 , letter2 , letter3 = trigram
          trigram_count[letter1 + letter2 + letter3] += 1

        # fourgram +  Fivegram

        fourgram_word = word + '####'
        fivegram_word = word + '####'

        fourgram_list = zip(fourgram_word[:-3] , fourgram_word[1:-2] , fourgram_word[2:-1] , fourgram_word[3:])

        for fourgram in fourgram_list:
          letter1 , letter2 , letter3 , letter4 = fourgram

          fourgram_count[letter1+letter2+letter3+letter4] += 1

        fivegram_list = zip(fivegram_word[:-4] , fivegram_word[1:-3] , fivegram_word[2:-2] , fivegram_word[3:-1] , fivegram_word[4:])

        for fivegram in fivegram_list:

          letter1 , letter2 , letter3 , letter4 , letter5 = fivegram

          fivegram_count[letter1 + letter2 + letter3 + letter4 + letter5] += 1


      return unigram_count , bigram_count , trigram_count , fourgram_count , fivegram_count


    def unigram_prob_forward(self , letter , unigram_count_forward):

      return (unigram_count_forward[letter] + 1) / (float(sum(unigram_count_forward.values())) + 250000)


    def bigram_prob_forward(self , letter1 , letter2 , bigram_count_forward , unigram_count_forward):

      return (bigram_count_forward[letter1 + letter2] + 1) / (unigram_count_forward[letter1] + 250000)

    def trigram_prob_forward(self, letter1 , letter2 , letter3 , trigram_count_forward , bigram_count_forward):

      return (trigram_count_forward[letter1 + letter2 + letter3] + 1) / (bigram_count_forward[letter1 + letter2] + 250000)

    def fourgram_prob_forward(self, letter1 , letter2 , letter3 , letter4 , fourgram_count_forward , trigram_count_forward):

      return (fourgram_count_forward[letter1 + letter2 + letter3 + letter4] + 1) / (trigram_count_forward[letter1 + letter2 + letter3] + 250000)

    def fivegram_prob_forward(self, letter1 , letter2 , letter3, letter4 , letter5 , fivegram_count_forward , fourgram_count_forward):

      return (fivegram_count_forward[letter1 + letter2 + letter3 + letter4 + letter5] + 1) / (fourgram_count_forward[letter1 + letter2 + letter3 + letter4] + 250000)



    def unigram_prob_backward(self , letter , unigram_count_backward):

      return (unigram_count_backward[letter] + 1) / (float(sum(unigram_count_backward.values())) + 250000)

    def bigram_prob_backward(self , letter1 , letter2 , bigram_count_backward , unigram_count_backward):

      return (bigram_count_backward[(letter1 + letter2)[::-1]] + 1) / (unigram_count_backward[letter1] + 250000)

    def trigram_prob_backward(self, letter1 , letter2 , letter3 , trigram_count_backward , bigram_count_backward):

      return (trigram_count_backward[(letter1 + letter2 + letter3)[::-1]] + 1) / (bigram_count_backward[(letter1 + letter2)[::-1]] + 250000)

    def fourgram_prob_backward(self, letter1 , letter2 , letter3 , letter4 , fourgram_count_backward , trigram_count_backward):

      return (fourgram_count_backward[(letter1 + letter2 + letter3 + letter4)[::-1]] + 1) / (trigram_count_backward[(letter1 + letter2 + letter3)[::-1]] + 250000)

    def fivegram_prob_backward(self, letter1 , letter2 , letter3, letter4 , letter5 , fivegram_count_backward , fourgram_count_backward):

      return (fivegram_count_backward[(letter1 + letter2 + letter3 + letter4 + letter5)[::-1]] + 1) / (fourgram_count_backward[(letter1 + letter2 + letter3 + letter4)[::-1]] + 250000)



    def guess(self, mask , counts_forward , counts_backward): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################
        # mask = str(mask)

        ## Make unigram corpus , bigram corpus , trigram corpus , fourgram corpus , fivegram corpus
        ## make unigram model, bigram

        mask = mask.split(" ")
        mask = "".join(mask)

        unigram_count_forward = counts_forward[0]
        bigram_count_forward = counts_forward[1]
        trigram_count_forward = counts_forward[2]
        fourgram_count_forward = counts_forward[3]
        fivegram_count_forward = counts_forward[4]



        unigram_count_backward = counts_backward[0]
        bigram_count_backward = counts_backward[1]
        trigram_count_backward = counts_backward[2]
        fourgram_count_backward = counts_backward[3]
        fivegram_count_backward = counts_backward[4]


        available = list(set(string.ascii_lowercase) - set(self.guessed_letters))
        # print(available)
        all_char_prob = []

        if mask.count("_") > 2:
          clean_word = mask.replace("_",".")

          # find length of passed word
          len_word = len(clean_word)

          # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
          current_dictionary = self.current_dictionary
          new_dictionary = []

          # iterate through all of the words in the old plausible dictionary
          for dict_word in current_dictionary:
              # continue if the word is not of the appropriate length
              if len(dict_word) != len_word:
                  continue

              # if dictionary word is a possible match then add it to the current dictionary
              if re.search(clean_word,dict_word):
                  new_dictionary.append(dict_word)

          # overwrite old possible words dictionary with updated version
          self.current_dictionary = new_dictionary


          # count occurrence of all characters in possible word matches
          full_dict_string = "".join(new_dictionary)

          c = collections.Counter(full_dict_string)
          sorted_letter_count = c.most_common()

          guess_letter = '!'

          # return most frequently occurring letter in all possible words that hasn't been guessed yet
          for letter,instance_count in sorted_letter_count:
              if letter not in self.guessed_letters:
                  guess_letter = letter
                  break

          # if no word matches in training dictionary, default back to ordering of full dictionary
          if guess_letter == '!':
              sorted_letter_count = self.full_dictionary_common_letter_sorted
              for letter,instance_count in sorted_letter_count:
                  if letter not in self.guessed_letters:
                      guess_letter = letter
                      break

          return guess_letter


        alpha1 = 0.1
        alpha2 = 0.15
        alpha3 = 0.2
        alpha4 = 0.25
        alpha5 = 0.3

        for char in available:

          # print(char)
          char_prob_forward = 1
          sum_prob_forward = 0

          char_prob_backward = 1
          sum_prob_backward = 0

          weight_forward = 0.3
          weight_backward = 0.7

          to_skip = 0
          if char in self.guessed_letters:
            continue

          # Forward
          for i in range(len(mask)):

            # Cases to handle

            if (mask[i] != '_'):
              to_skip += 1
              continue
            # First Letter

            if (i == 0) and (mask[i] == '_'):

              char_prob_forward *= alpha5 * self.fivegram_prob_forward('#' , '#' , '#' , '#' , char , fivegram_count_forward , fourgram_count_forward)
              sum_prob_forward += alpha5 * self.fivegram_prob_forward('#' , '#' , '#' , '#' , char , fivegram_count_forward , fourgram_count_forward)

            # Second Letter
            elif (i == 1):
              if (mask[i-1] != '_'):
                char_prob_forward *= alpha5 * self.fivegram_prob_forward('#' , '#' , '#' , mask[i-1] , char , fivegram_count_forward , fourgram_count_forward)
                sum_prob_forward +=alpha5 * self.fivegram_prob_forward('#' , '#' , '#' , mask[i-1] , char , fivegram_count_forward , fourgram_count_forward)

              else:
                char_prob_forward *= alpha1 * self.unigram_prob_forward(char , unigram_count_forward)
                sum_prob_forward += alpha1 * self.unigram_prob_forward(char , unigram_count_forward)

            # Third Letter
            elif (i == 2):
              if (mask[i-1] != '_') and (mask[i-2] != '_'):
                char_prob_forward *= alpha5 * self.fivegram_prob_forward('#' , '#' , mask[i-2] , mask[i-1] , char , fivegram_count_forward , fourgram_count_forward)
                sum_prob_forward += alpha5 * self.fivegram_prob_forward('#' , '#' , mask[i-2] , mask[i-1] , char , fivegram_count_forward , fourgram_count_forward)

              elif(mask[i-1] == '_'):
                char_prob_forward *= alpha1 * self.unigram_prob_forward(char , unigram_count_forward)
                sum_prob_forward += alpha1 * self.unigram_prob_forward(char , unigram_count_forward)

              elif(mask[i-1] != '_') and (mask[i-2] == '_'):
                char_prob_forward *= alpha2 * self.bigram_prob_forward(mask[i-1] , char , bigram_count_forward , unigram_count_forward)
                sum_prob_forward += alpha2 * self.bigram_prob_forward(mask[i-1] , char , bigram_count_forward , unigram_count_forward)

            # fourth letter
            elif (i == 3):
              if(mask[i-1] != '_') and (mask[i-2] != '_') and (mask[i-3] != '_'):
                char_prob_forward *= alpha5 * self.fivegram_prob_forward('#' , mask[i-3] , mask[i-2] , mask[i-1] , char , fivegram_count_forward , fourgram_count_forward)
                sum_prob_forward += alpha5 * self.fivegram_prob_forward('#' , mask[i-3] , mask[i-2] , mask[i-1] , char , fivegram_count_forward , fourgram_count_forward)

              elif(mask[i-1] == '_'):
                char_prob_forward *= alpha1 * self.unigram_prob_forward(char , unigram_count_forward)
                sum_prob_forward += alpha1 * self.unigram_prob_forward(char , unigram_count_forward)

              elif(mask[i-1] != '_') and (mask[i-2] == '_'):
                char_prob_forward *= alpha2 * self.bigram_prob_forward(mask[i-1] , char , bigram_count_forward , unigram_count_forward)
                sum_prob_forward += alpha2 * self.bigram_prob_forward(mask[i-1] , char , bigram_count_forward , unigram_count_forward)

              elif(mask[i-1] != '_') and (mask[i-2] != '_') and (mask[i-3] == '_'):
                char_prob_forward *= alpha3 * self.trigram_prob_forward(mask[i-2] , mask[i-1] , char , trigram_count_forward , bigram_count_forward)
                sum_prob_forward += alpha3 * self.trigram_prob_forward(mask[i-2] , mask[i-1] , char , trigram_count_forward , bigram_count_forward)

            # Fifth letter

            else:
              if (mask[i-1] != '_') and (mask[i-2] != '_') and (mask[i-3] != '_') and (mask[i-4] != '_'):
                char_prob_forward *= alpha5 * self.fivegram_prob_forward(mask[i-4] , mask[i-3] , mask[i-2] , mask[i-1] , char , fivegram_count_forward , fourgram_count_forward)
                sum_prob_forward += alpha5 * self.fivegram_prob_forward(mask[i-4] , mask[i-3] , mask[i-2] , mask[i-1] , char , fivegram_count_forward , fourgram_count_forward)

              elif(mask[i-1] == '_'):

                char_prob_forward *= alpha1 * self.unigram_prob_forward(char , unigram_count_forward)
                sum_prob_forward += alpha1 * self.unigram_prob_forward(char , unigram_count_forward)

              elif(mask[i-1] != '_') and (mask[i-2] == '_'):

                char_prob_forward *= alpha2 * self.bigram_prob_forward(mask[i-1] , char , bigram_count_forward , unigram_count_forward)
                sum_prob_forward += alpha2 * self.bigram_prob_forward(mask[i-1] , char , bigram_count_forward , unigram_count_forward)

              elif(mask[i-1] != '_') and (mask[i-2] != '_') and (mask[i-3] == '_'):
                char_prob_forward *= alpha3 * self.trigram_prob_forward(mask[i-2] , mask[i-1] , char , trigram_count_forward , bigram_count_forward)
                sum_prob_forward += alpha3 * self.trigram_prob_forward(mask[i-2] , mask[i-1] , char , trigram_count_forward , bigram_count_forward)

              elif(mask[i-1] != '_') and (mask[i-2] != '_') and (mask[i-3] != '_') and (mask[i-4] == '_'):
                char_prob_forward *= alpha4 * self.fourgram_prob_forward(mask[i-3] , mask[i-2] , mask[i-1] , char , fourgram_count_forward , trigram_count_forward)
                sum_prob_forward += alpha4 * self.fourgram_prob_forward(mask[i-3] , mask[i-2] , mask[i-1] , char , fourgram_count_forward , trigram_count_forward)


          # Backward
          for i in reversed(range(len(mask))):


            if(mask[i] != '_'):
              continue
            n = (len(mask) - 1)
            # Cases to handle

            # First Letter

            if (i == (n)) and (mask[i] == '_'):

              char_prob_backward *= alpha5 * self.fivegram_prob_backward('#' , '#' , '#' , '#' , char , fivegram_count_backward , fourgram_count_backward)
              sum_prob_backward += alpha5 * self.fivegram_prob_backward('#' , '#' , '#' , '#' , char , fivegram_count_backward , fourgram_count_backward)


            # Second Letter
            elif (i == (n-1)):
              if (mask[i+1] != '_'):
                char_prob_backward *= alpha5 * self.fivegram_prob_backward('#' , '#' , '#' , mask[i+1] , char , fivegram_count_backward , fourgram_count_backward)
                sum_prob_backward +=alpha5 * self.fivegram_prob_backward('#' , '#' , '#' , mask[i+1] , char , fivegram_count_backward , fourgram_count_backward)


              else:
                char_prob_backward *= alpha1 * self.unigram_prob_backward(char , unigram_count_backward)
                sum_prob_backward += alpha1 * self.unigram_prob_backward(char , unigram_count_backward)


            # Third Letter
            elif (i == (n-2)):
              if (mask[i+1] != '_') and (mask[i+2] != '_'):
                char_prob_backward *= alpha5 * self.fivegram_prob_backward('#' , '#' , mask[i+2] , mask[i+1] , char , fivegram_count_backward , fourgram_count_backward)
                sum_prob_backward += alpha5 * self.fivegram_prob_backward('#' , '#' , mask[i+2] , mask[i+1] , char , fivegram_count_backward , fourgram_count_backward)


              elif(mask[i+1] == '_'):
                char_prob_backward *= alpha1 * self.unigram_prob_backward(char , unigram_count_backward)
                sum_prob_backward += alpha1 * self.unigram_prob_backward(char , unigram_count_backward)


              elif(mask[i+1] != '_') and (mask[i+2] == '_'):
                char_prob_backward *= alpha2 * self.bigram_prob_backward(mask[i-1] , char , bigram_count_backward , unigram_count_backward)
                sum_prob_backward += alpha2 * self.bigram_prob_backward(mask[i-1] , char , bigram_count_backward , unigram_count_backward)


            # fourth letter
            elif (i == (n-3)):
              if(mask[i+1] != '_') and (mask[i+2] != '_') and (mask[i+3] != '_'):
                char_prob_backward *= alpha5 * self.fivegram_prob_backward('#' , mask[i+3] , mask[i+2] , mask[i+1] , char , fivegram_count_backward , fourgram_count_backward)
                sum_prob_backward += alpha5 * self.fivegram_prob_backward('#' , mask[i+3] , mask[i+2] , mask[i+1] , char , fivegram_count_backward , fourgram_count_backward)


              elif(mask[i+1] == '_'):
                char_prob_backward *= alpha1 * self.unigram_prob_backward(char , unigram_count_backward)
                sum_prob_backward += alpha1 * self.unigram_prob_backward(char , unigram_count_backward)


              elif(mask[i+1] != '_') and (mask[i+2] == '_'):
                char_prob_backward *= alpha2 * self.bigram_prob_backward(mask[i+1] , char , bigram_count_backward , unigram_count_backward)
                sum_prob_backward += alpha2 * self.bigram_prob_backward(mask[i+1] , char , bigram_count_backward , unigram_count_backward)


              elif(mask[i+1] != '_') and (mask[i+2] != '_') and (mask[i+3] == '_'):
                char_prob_backward *= alpha3 * self.trigram_prob_backward(mask[i+2] , mask[i+1] , char , trigram_count_backward , bigram_count_backward)
                sum_prob_backward += alpha3 * self.trigram_prob_backward(mask[i+2] , mask[i+1] , char , trigram_count_backward , bigram_count_backward)


            # Fifth letter

            else:
              if (mask[i+1] != '_') and (mask[i+2] != '_') and (mask[i+3] != '_') and (mask[i+4] != '_'):
                char_prob_backward *= alpha5 * self.fivegram_prob_backward(mask[i+4] , mask[i+3] , mask[i+2] , mask[i+1] , char , fivegram_count_backward , fourgram_count_backward)
                sum_prob_backward += alpha5 * self.fivegram_prob_backward(mask[i+4] , mask[i+3] , mask[i+2] , mask[i+1] , char , fivegram_count_backward , fourgram_count_backward)


              elif(mask[i+1] == '_'):

                char_prob_backward *= alpha1 * self.unigram_prob_backward(char , unigram_count_backward)
                sum_prob_backward += alpha1 * self.unigram_prob_backward(char , unigram_count_backward)


              elif(mask[i+1] != '_') and (mask[i+2] == '_'):

                char_prob_backward *= alpha2 * self.bigram_prob_backward(mask[i+1] , char , bigram_count_backward , unigram_count_backward)
                sum_prob_backward += alpha2 * self.bigram_prob_backward(mask[i+1] , char , bigram_count_backward , unigram_count_backward)


              elif(mask[i+1] != '_') and (mask[i+2] != '_') and (mask[i+3] == '_'):
                char_prob_backward *= alpha3 * self.trigram_prob_backward(mask[i+2] , mask[i+1] , char , trigram_count_backward , bigram_count_backward)
                sum_prob_backward += alpha3 * self.trigram_prob_backward(mask[i+2] , mask[i+1] , char , trigram_count_backward , bigram_count_backward)


              elif(mask[i+1] != '_') and (mask[i+2] != '_') and (mask[i+3] != '_') and (mask[i+4] == '_'):
                char_prob_backward *= alpha4 * self.fourgram_prob_backward(mask[i+3] , mask[i+2] , mask[i+1] , char , fourgram_count_backward , trigram_count_backward)
                sum_prob_backward += alpha4 * self.fourgram_prob_backward(mask[i+3] , mask[i+2] , mask[i+1] , char , fourgram_count_backward , trigram_count_backward)



          all_char_prob.append((char_prob_forward / float(sum_prob_forward**(len(mask) - to_skip)))*  weight_forward + (char_prob_backward / float(sum_prob_backward**(len(mask) - to_skip)))*  weight_backward)

        return available[all_char_prob.index(max(all_char_prob))]




    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):

    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))

    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1

        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes

    if verbose:
        print('Out of guesses. The word was', secret_word)
    return mistakes


  def test_guesser(guesser, test=test_set):
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [ ]:
api = HangmanAPI()

In [ ]:
current_status = total_practice_successes

In [ ]:
counter = 0

In [ ]:
start = time.time()
for i in range(20):
  counter+=1
  api.test_guesser(api.guess())
  time.sleep(3)

end = time.time()
print("Total Time : " , (end - start)/60)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = (total_practice_successes / total_practice_runs) * 100
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))
print("Current_practice rate : " , ((total_practice_successes - current_status) / counter) * 100)
print("Total Games played : " , counter)